In [3]:
import json

with open('../data/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [ ]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)


In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-mpnet-base-v2")

In [ ]:
operations = []

for doc in documents:
    # Encoding the 'text' field
    doc['text_vector'] = model.encode(doc['text']).tolist()
    operations.append(doc)

In [ ]:
operations[0]

In [30]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

es_client.info()

ObjectApiResponse({'name': 'be306b791aff', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'O-CxRtx8RdqjyMnH3kqG9w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [33]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [34]:
index_name = 'course-questions'

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [36]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [42]:
search_term = "When can I join the data engineering course?"
vector_search_term = model.encode(search_term)

In [43]:
query = {
    "field" : "text_vector",
    "query_vector" : vector_search_term,
    "k" : 5,
    "num_candidates" : 10000
}

In [44]:
res = es_client.search(index=index_name, knn=query, source=["text","section","question","course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'eeO62JABiPMUq5jPdKb-',
  '_score': 0.7747749,
  '_source': {'question': 'Course - When will the course start?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel."}},
 {'_index': 'course-questions',
  '_id': 'luO62JABiPMUq5jPvqjE',
  '_score': 0.7747749,
  '_source': {'question': 'Course - When will the course start?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': "The purpose of 